In [3]:
from dash import Dash, dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import os
from IPython.display import clear_output

# ✅ Tell Dash to look for assets one level up
app = JupyterDash(
    __name__,
    assets_folder=os.path.abspath("../assets"),
    assets_url_path="/assets"
)

app.title = "🕊️ Bird Carousel"

# 📦 Liste des espèces d'oiseaux
BIRD_SPECIES = sorted([
    d for d in os.listdir("../assets/birds")
    if os.path.isdir(os.path.join("../assets/birds", d))
])

# 📦 Dictionnaire {species: [images]}
bird_images = {}
for species in BIRD_SPECIES:
    folder = os.path.join("../assets/birds", species)
    images = []
    for root, _, files in os.walk(folder):
        images += [os.path.join(root, f) for f in files if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    bird_images[species] = sorted(images)

# 🎨 Layout principal
app.layout = html.Div([
    html.H2("\ud83d\udd4a\ufe0f Bird Carousel", style={
        "textAlign": "center", "marginBottom": "25px", "marginTop": "10px",
        "fontSize": "28px", "fontWeight": "bold", "fontFamily": "Georgia, serif"
    }),

    html.Div([
        html.Label("Select bird species:", style={"fontSize": "16px", "marginBottom": "6px", "fontFamily": "Georgia, serif"}),
        dcc.Dropdown(
            id='species-dropdown',
            options=[{"label": s.capitalize(), "value": s} for s in BIRD_SPECIES],
            value=BIRD_SPECIES[0],
            clearable=False,
            style={"width": "300px", "margin": "0 auto", "fontFamily": "Georgia, serif"}
        )
    ], style={"textAlign": "center", "marginBottom": "30px"}),

    html.Div(id='carousel-container', style={"textAlign": "center"})
], style={"fontFamily": "Georgia, serif", "padding": "30px"})


# 🔄 Affiche le bon carousel pour chaque espèce
@app.callback(
    Output('carousel-container', 'children'),
    Input('species-dropdown', 'value')
)
def update_carousel(species):
    images = bird_images.get(species, [])
    if not images:
        return html.P("No images available.", style={"textAlign": "center"})

    filename = os.path.basename(images[0])

    return html.Div([
        html.Div([
            html.Button("\u2b05\ufe0f", id="prev-button", n_clicks=0,
                        style={"fontSize": "18px", "padding": "5px 10px", "marginRight": "15px"}),
            html.Img(id="carousel-image", src=images[0],
                     style={"height": "450px", "display": "inline-block", "verticalAlign": "middle"}),
            html.Button("\u27a1\ufe0f", id="next-button", n_clicks=0,
                        style={"fontSize": "18px", "padding": "5px 10px", "marginLeft": "15px"}),
        ], style={"display": "flex", "justifyContent": "center", "alignItems": "center", "marginBottom": "20px"}),

        dcc.Store(id='image-index', data=0),
        dcc.Store(id='current-images', data=images),
        html.Div(f"1/{len(images)} - {filename}", id='filename-display',
                 style={"textAlign": "center", "fontSize": "14px", "fontFamily": "Georgia, serif", "color": "#444"})
    ])

# 🔄 Navigation entre les images
@app.callback(
    Output("carousel-image", "src"),
    Output("image-index", "data"),
    Output("filename-display", "children"),
    Input("prev-button", "n_clicks"),
    Input("next-button", "n_clicks"),
    State("current-images", "data"),
    State("image-index", "data"),
    prevent_initial_call=True
)
def navigate_images(prev_clicks, next_clicks, images, index):
    if not images:
        return "", 0, "No images"

    triggered = ctx.triggered_id
    if triggered == "next-button":
        index = (index + 1) % len(images)
    elif triggered == "prev-button":
        index = (index - 1) % len(images)

    filename = os.path.basename(images[index])
    return images[index], index, f"{index + 1}/{len(images)} - {filename}"

# 🚀 Lancer (changer le port si besoin)
if __name__ == "__main__":
    clear_output(wait=True)
    app.run(port=8130)
